In [1]:
import requests

# Specify the URL of the page you want to download
url = "https://pokemondb.net/pokedex/all"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to an HTML file
    with open("pokemon.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Page saved as 'Pokemon.html'")
else:
    print(f"Failed to download page. Status code: {response.status_code}")

Page saved as 'Pokemon.html'


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#URL der Webseite
url = "https://pokemondb.net/pokedex/all"

#HTTP-Anfrage senden
response = requests.get(url)
response.raise_for_status()  # Fehler auslösen, falls die Anfrage fehlschlägt

#HTML-Inhalt der Seite parsen
soup = BeautifulSoup(response.text, 'html.parser')

#Tabelle finden
table = soup.find('table', {'id': 'pokedex'})

#Tabellenüberschriften extrahieren
headers = [header.text for header in table.find('thead').find_all('th')]

#Tabelleninhalte extrahieren
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all(['th', 'td'])
    rows.append([cell.text.strip() for cell in cells])

#DataFrame erstellen
df = pd.DataFrame(rows, columns=headers)

#Ausgabe des DataFrames
print(df.head())

#Optional: DataFrame in eine CSV-Datei speichern
df.to_csv("pokemon_data.csv", index=False)

      #                    Name          Type Total  HP Attack Defense  \
0  0001               Bulbasaur  Grass Poison   318  45     49      49   
1  0002                 Ivysaur  Grass Poison   405  60     62      63   
2  0003                Venusaur  Grass Poison   525  80     82      83   
3  0003  Venusaur Mega Venusaur  Grass Poison   625  80    100     123   
4  0004              Charmander          Fire   309  39     52      43   

  Sp. Atk Sp. Def Speed  
0      65      65    45  
1      80      80    60  
2     100     100    80  
3     122     120    80  
4      60      50    65  


In [10]:
import pandas as pd

# CSV-Datei einlesen
df = pd.read_csv('pokemon_data.csv')

# Gesamtwert bereits in der CSV enthalten
# Umbenennen von Spalten für einfacheren Zugriff
df.rename(columns={
    'Type': 'Typ',
    'Total': 'Gesamt',
    'HP': 'KP',
    'Attack': 'Angriff',
    'Defense': 'Verteidigung',
    'Sp. Atk': 'SpezAng',
    'Sp. Def': 'SpezVert',
    'Speed': 'Initiative'
}, inplace=True)

# Funktion: Stärkstes Pokémon pro Typ
def strongest_per_type(df):
    types = {}
    for _, row in df.iterrows():
        for t in row['Typ'].split():  # Annahme: Typen sind durch Leerzeichen getrennt
            if t not in types or row['Gesamt'] > types[t]['Gesamt']:
                types[t] = row
    return pd.DataFrame(types.values())

# Funktion: Beste Angreifer
def best_attackers(df, top_n=5):
    return df.sort_values(by='Angriff', ascending=False).head(top_n)

# Funktion: Durchschnittswerte pro Typ
def average_stats_per_type(df):
    type_stats = {}
    for _, row in df.iterrows():
        for t in row['Typ'].split():
            if t not in type_stats:
                type_stats[t] = []
            type_stats[t].append(row[['KP', 'Angriff', 'Verteidigung', 'SpezAng', 'SpezVert', 'Initiative']])
    averages = {t: pd.DataFrame(stats).mean().to_dict() for t, stats in type_stats.items()}
    return pd.DataFrame(averages).T

# Ergebnisse
print("Stärkstes Pokémon pro Typ:")
print(strongest_per_type(df))

print("\nBeste Angreifer:")
print(best_attackers(df))

print("\nDurchschnittswerte pro Typ:")
print(average_stats_per_type(df))


Stärkstes Pokémon pro Typ:
        #                      Name               Typ  Gesamt   KP  Angriff  \
319   254    Sceptile Mega Sceptile      Grass Dragon     630   70      110   
1058  890       Eternatus Eternamax     Poison Dragon    1125  255      115   
473   383    Groudon Primal Groudon       Ground Fire     770  100      180   
475   384    Rayquaza Mega Rayquaza     Dragon Flying     780  105      180   
1058  890       Eternatus Eternamax     Poison Dragon    1125  255      115   
471   382      Kyogre Primal Kyogre             Water     770  100      150   
168   127        Pinsir Mega Pinsir        Bug Flying     600   65      155   
605   493                    Arceus            Normal     720  120      120   
312   248  Tyranitar Mega Tyranitar         Rock Dark     700  100      164   
772   644                    Zekrom   Dragon Electric     680  100      150   
201   150      Mewtwo Mega Mewtwo X  Psychic Fighting     780  106      190   
473   383    Groudon Prim